In [16]:
#| default_exp routes.openai

# OpenAI Routes
> Default description (change me)

In [17]:
#| export
from openai import AsyncOpenAI


In [18]:
#| exporti
from dataclasses import dataclass
from typing import Union, Dict, List, Literal

import json
import os

from openai import AsyncClient as AsyncOpenaiClient

from agent_mafia.client.ResponseGetData import ResponseGetDataOpenAi


In [19]:
from dotenv import load_dotenv

In [20]:
assert load_dotenv()
model = "gpt-4o-mini-2024-07-18"

In [21]:
#| export

def generate_openai_client(
    api_key: str,
    base_url: str = None,
    is_ollama: bool = False
) -> AsyncOpenaiClient:

    if is_ollama:
        return AsyncOpenaiClient(
            api_key=api_key,
            base_url=base_url,
        )

    return AsyncOpenaiClient(api_key=api_key)

In [22]:
#| export

@dataclass
class ChatMessage:
    """Format of messages setn to the browser/API"""

    role: Literal["user", "model"]
    content: str
    timestamp: str = None

    def to_json(self):
        return {
            "role": self.role,
            "content": self.content,
            "timestamp": self.timestamp
        }

async def generate_openai_chat(
    async_client: AsyncOpenAI,
    messages : List[ChatMessage],
    model: str = None, # llm model to use
    response_format: Union[Dict[str, str], None] = None,
    return_raw: bool = False,
):
    clean_message = [ msg.to_json() if isinstance(msg, ChatMessage) else msg for msg in messages]

    res = await async_client.chat.completions.create(
        model=model, messages=clean_message, response_format=response_format
    )

    rgd = ResponseGetDataOpenAi.from_res(res)

    if return_raw:
        return rgd

    content = res.choices[0].message.content
    rgd.response = content

    if response_format and response_format.get("type") == "json_object":
        rgd.response = json.loads(content)

    return rgd

In [23]:
openai_client = generate_openai_client(
    api_key = os.environ.get('OPENAI_API_KEY'),
    )

(await generate_openai_chat(
        async_client = openai_client,
        messages = [ChatMessage(role = 'user', content = 'hello, my name is jae', )],
        model = model, 
        return_raw = False)).response


'Hello, Jae! How can I assist you today?'

In [24]:
#| export
async def generate_openai_embedding(
    text,
    async_client: AsyncOpenAI,
    model="text-embedding-3-small",
    return_raw: bool = False,
    debug_prn: bool = False,
) -> List[float]:

    if debug_prn:
        print("📚 - starting LLM")

    async_client = async_client

    res = await async_client.embeddings.create(model=model, input=text)

    if return_raw:
        return res

    return res.data[0].embedding

In [25]:
await generate_openai_embedding(
    text = "hello world",
    async_client = openai_client
)


[-0.006686408072710037,
 -0.039188627153635025,
 0.03413885831832886,
 0.028741348534822464,
 -0.02476503513753414,
 -0.04197053611278534,
 -0.030253255739808083,
 0.049257926642894745,
 -0.013879304751753807,
 -0.017689310014247894,
 0.015368533320724964,
 -0.027017775923013687,
 -0.020955029875040054,
 -0.027819085866212845,
 0.00863298773765564,
 0.03562052547931671,
 -0.053642455488443375,
 -0.0023396757896989584,
 0.00882953591644764,
 0.048018161207437515,
 0.03713243082165718,
 -0.00918483454734087,
 -0.008769059553742409,
 0.011392218060791492,
 0.014128769747912884,
 -0.0021941547747701406,
 -0.03746505081653595,
 0.04538744315505028,
 0.01124858669936657,
 -0.03970267251133919,
 0.023389199748635292,
 -0.05058840289711952,
 0.012019659392535686,
 -5.646027057082392e-05,
 0.0160262119024992,
 0.006115663331001997,
 0.031931471079587936,
 0.003422579262405634,
 -0.008640547282993793,
 -0.010598466731607914,
 -0.03734409809112549,
 -0.03447147458791733,
 0.05001387745141983,
 0.

In [26]:
#| hide
nbdev.nbdev_export('/openai.ipynb')

NameError: name 'nbdev' is not defined